# Projet 4 : Anticiper les besoins en consommation électrique de bâtiments
*Pierre-Eloi Ragetly*

Ce projet fait parti du parcours *DataScientist* d'OpenClassrooms.

L'objectif principal est de trouver un modèle permettant de prédire **les émissions de CO2 et la consommation totale d’énergie de bâtiments non destinés à l'habitation.**

Pour cela nous disposons des données de la ville de Seattle pour les années 2015 et 2016. Ces données sont à récupérer sur le site kaggle.

# Partie I : Data Wrangling

Ce notebook a pour but de présenter les données et de décrire les opérations nécessaire au nettoyage afin d'obtenir un format adapté à l'analyse et la modélisation.

In [43]:
# Import des librairies usuelles
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import pandas as pd
import seaborn as sns

In [44]:
# Change some default parameters of matplotlib using seaborn
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'axes.titleweight': 'bold'})
sns.set(style='ticks')
current_palette = sns.color_palette('RdBu')
sns.set_palette(current_palette)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Récupération-des-données" data-toc-modified-id="Récupération-des-données-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Récupération des données</a></span></li><li><span><a href="#Comparaison-des-deux-jeux-de-données" data-toc-modified-id="Comparaison-des-deux-jeux-de-données-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Comparaison des deux jeux de données</a></span><ul class="toc-item"><li><span><a href="#Variables-n'étant-pas-présentes-dans-l'un-des-jeux-de-données" data-toc-modified-id="Variables-n'étant-pas-présentes-dans-l'un-des-jeux-de-données-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Variables n'étant pas présentes dans l'un des jeux de données</a></span></li><li><span><a href="#Traitement-de-la-variable-localisation" data-toc-modified-id="Traitement-de-la-variable-localisation-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Traitement de la variable localisation</a></span></li><li><span><a href="#Renommer-les-variables-de-2015-comme-celles-de-2016" data-toc-modified-id="Renommer-les-variables-de-2015-comme-celles-de-2016-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Renommer les variables de 2015 comme celles de 2016</a></span></li><li><span><a href="#Variables-non-présentes-dans-le-jeu-de-données-2016" data-toc-modified-id="Variables-non-présentes-dans-le-jeu-de-données-2016-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Variables non présentes dans le jeu de données 2016</a></span></li><li><span><a href="#Ajout-des-variables-manquantes-pour-le-jeu-de-données-de-2016" data-toc-modified-id="Ajout-des-variables-manquantes-pour-le-jeu-de-données-de-2016-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Ajout des variables manquantes pour le jeu de données de 2016</a></span></li><li><span><a href="#Traitement-de-la-variable-OtherFuelUse(kBtu)" data-toc-modified-id="Traitement-de-la-variable-OtherFuelUse(kBtu)-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Traitement de la variable OtherFuelUse(kBtu)</a></span></li></ul></li><li><span><a href="#Sélection-des-observations" data-toc-modified-id="Sélection-des-observations-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Sélection des observations</a></span><ul class="toc-item"><li><span><a href="#Vérifier-la-présence-de-doublons" data-toc-modified-id="Vérifier-la-présence-de-doublons-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Vérifier la présence de doublons</a></span></li><li><span><a href="#Traiter-les-outliers" data-toc-modified-id="Traiter-les-outliers-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Traiter les outliers</a></span></li><li><span><a href="#Écarter-les-bâtiments-destinés-à-l'habitation" data-toc-modified-id="Écarter-les-bâtiments-destinés-à-l'habitation-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Écarter les bâtiments destinés à l'habitation</a></span></li></ul></li><li><span><a href="#Nettoyage-des-variables-liés-à-la-consommation" data-toc-modified-id="Nettoyage-des-variables-liés-à-la-consommation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Nettoyage des variables liés à la consommation</a></span><ul class="toc-item"><li><span><a href="#Vérifier-que-les-consommations-sont-bien-positives" data-toc-modified-id="Vérifier-que-les-consommations-sont-bien-positives-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Vérifier que les consommations sont bien positives</a></span></li><li><span><a href="#Vérifier-les-conversions" data-toc-modified-id="Vérifier-les-conversions-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Vérifier les conversions</a></span></li><li><span><a href="#Vérification-des-totaux" data-toc-modified-id="Vérification-des-totaux-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Vérification des totaux</a></span></li></ul></li><li><span><a href="#Nettoyage-des-variables-liées-au-Gross-Floor-Area-(GFA)" data-toc-modified-id="Nettoyage-des-variables-liées-au-Gross-Floor-Area-(GFA)-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Nettoyage des variables liées au Gross Floor Area (GFA)</a></span></li></ul></div>

## Récupération des données

Nous commençons par télécharger les données du site *kaggle*. Ces données sont contenus dans 2 fichier csv :
- 2015-building-energy-benchmarking
- 2016-building-energy-benchmarking

Notre but étant de prédire des données de consommation, avoir des données pour deux dates différentes est un gros plus. **Nous allons pouvoir utiliser les donnée de 2015 (l'année la plus ancienne) comme jeu d'entraînement et les données de 2016 (les plus récentes) comme jeu de test.** Les données étant déja contenues dans deux jeux de données distincts, nous n'aurons pas à séparer les données en deux (test et entraînement).

Une fois téléchargées, nous pouvons les charger dans des DataFrame.

In [45]:
raw_data_2015 = pd.read_csv('data/2015-building-energy-benchmarking.csv')
print(raw_data_2015.shape)
raw_data_2015.head()

(3340, 47)


,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,TaxParcelIdentificationNumber,Location,CouncilDistrictCode,Neighborhood,YearBuilt,...,GHGEmissionsIntensity(kgCO2e/ft2),DefaultData,Comment,ComplianceStatus,Outlier,2010 Census Tracts,Seattle Police Department Micro Community Policing Plan Areas,City Council Districts,SPD Beats,Zip Codes
0,1,2015,NonResidential,Hotel,MAYFLOWER PARK HOTEL,659000030,"{'latitude': '47.61219025', 'longitude': '-122...",7,DOWNTOWN,1927,...,2.64,No,NaN,Compliant,NaN,NaN,14.0,NaN,31.0,18081
1,2,2015,NonResidential,Hotel,PARAMOUNT HOTEL,659000220,"{'latitude': '47.61310583', 'longitude': '-122...",7,DOWNTOWN,1996,...,2.38,No,NaN,Compliant,NaN,NaN,14.0,NaN,31.0,18081
2,3,2015,NonResidential,Hotel,WESTIN HOTEL,659000475,"{'latitude': '47.61334897', 'longitude': '-122...",7,DOWNTOWN,1969,...,1.92,Yes,NaN,Compliant,NaN,NaN,56.0,NaN,31.0,18081
3,5,2015,NonResidential,Hotel,HOTEL MAX,659000640,"{'latitude': '47.61421585', 'longitude': '-122...",7,DOWNTOWN,1926,...,31.38,No,NaN,Compliant,High Outlier,NaN,56.0,NaN,31.0,18081
4,8,2015,NonResidential,Hotel,WARWICK SEATTLE HOTEL,659000970,"{'latitude': '47.6137544', 'longitude': '-122....",7,DOWNTOWN,1980,...,4.02,No,NaN,Compliant,NaN,NaN,56.0,NaN,31.0,19576


In [46]:
raw_data_2016 = pd.read_csv('data/2016-building-energy-benchmarking.csv')
print(raw_data_2016.shape)
raw_data_2016.head()

(3376, 46)


,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,...,Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
0,1,2016,NonResidential,Hotel,Mayflower park hotel,405 Olive way,Seattle,WA,98101.0,0659000030,...,1.156514e+06,3946027.0,12764.52930,1276453.0,False,NaN,Compliant,NaN,249.98,2.83
1,2,2016,NonResidential,Hotel,Paramount Hotel,724 Pine street,Seattle,WA,98101.0,0659000220,...,9.504252e+05,3242851.0,51450.81641,5145082.0,False,NaN,Compliant,NaN,295.86,2.86
2,3,2016,NonResidential,Hotel,5673-The Westin Seattle,1900 5th Avenue,Seattle,WA,98101.0,0659000475,...,1.451544e+07,49526664.0,14938.00000,1493800.0,False,NaN,Compliant,NaN,2089.28,2.19
3,5,2016,NonResidential,Hotel,HOTEL MAX,620 STEWART ST,Seattle,WA,98101.0,0659000640,...,8.115253e+05,2768924.0,18112.13086,1811213.0,False,NaN,Compliant,NaN,286.43,4.67
4,8,2016,NonResidential,Hotel,WARWICK SEATTLE HOTEL (ID8),401 LENORA ST,Seattle,WA,98121.0,0659000970,...,1.573449e+06,5368607.0,88039.98438,8803998.0,False,NaN,Compliant,NaN,505.01,2.88


## Comparaison des deux jeux de données

### Variables n'étant pas présentes dans l'un des jeux de données

Les deux jeux de données possédent un nombre d'observations similaires, en revanche, nous notons qu'ils ne possédent pas le même nombre de variables. Or, il est indispensable que les deux possédent exactement les mêmes caractéristiques pour pouvoir entraîner (sur les données de 2015) et tester (sur les données de 2016) notre modèle.

On va donc commencer par ne garder que les variables communes aux deux années.

In [47]:
features_2015 = raw_data_2015.columns.tolist()
features_2016 = raw_data_2016.columns.tolist()
features = [c for c in features_2016 if c in features_2015]
print(len(features))

37


Dix-neuf variables ne sont pas présentes dans l'un des deux jeux de données, cela représente plus de 30% de nos variables, c'est loin d'être négligeable. Avant de les écarter, regardons les en détail pour voir si elles ne contiennent pas d'éléments clés.

In [48]:
feat_to_del = [c for c in features_2015 if c not in features] + [c for c in features_2016 if c not in features]
print(len(feat_to_del))
print(feat_to_del)

19
['Location', 'OtherFuelUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)', 'GHGEmissionsIntensity(kgCO2e/ft2)', 'Comment', '2010 Census Tracts', 'Seattle Police Department Micro Community Policing Plan Areas', 'City Council Districts', 'SPD Beats', 'Zip Codes', 'Address', 'City', 'State', 'ZipCode', 'Latitude', 'Longitude', 'Comments', 'TotalGHGEmissions', 'GHGEmissionsIntensity']


Nous remarquons que certaines variables semblent très proches :
- Location & Latitude/Longitude
- GHGEmissions(MetricTonsCO2e) & TotalGHGEmissions
- GHGEmissionsIntensity(kgCO2e/ft2) & GHGEmissionsIntensity
- Zip Codes & ZipCode
- Comment & Comments

Commençons par vérifier si ces variables ne seraient pas les mêmes, mais avec des noms différents.

In [49]:
feat_to_check = ['Location', 'Latitude', 'Longitude',
                 'GHGEmissions(MetricTonsCO2e)', 'TotalGHGEmissions',
                 'GHGEmissionsIntensity(kgCO2e/ft2)', 'GHGEmissionsIntensity',
                 'Zip Codes', 'ZipCode',
                 'Comment', 'Comments']
key = 'OSEBuildingID'
feat_to_check += [key]

# keep only the features to be checked for each dataset
df_2015 = raw_data_2015.loc[:, raw_data_2015.columns.isin(feat_to_check)]
df_2016 = raw_data_2016.loc[:, raw_data_2016.columns.isin(feat_to_check)]

# use the feature 'OSEBuildingID' to merge datasets
df = pd.merge(df_2015, df_2016, on=key)
df.head()

,OSEBuildingID,Location,GHGEmissions(MetricTonsCO2e),GHGEmissionsIntensity(kgCO2e/ft2),Comment,Zip Codes,ZipCode,Latitude,Longitude,Comments,TotalGHGEmissions,GHGEmissionsIntensity
0,1,"{'latitude': '47.61219025', 'longitude': '-122...",249.43,2.64,NaN,18081,98101.0,47.61220,-122.33799,NaN,249.98,2.83
1,2,"{'latitude': '47.61310583', 'longitude': '-122...",263.51,2.38,NaN,18081,98101.0,47.61317,-122.33393,NaN,295.86,2.86
2,3,"{'latitude': '47.61334897', 'longitude': '-122...",2061.48,1.92,NaN,18081,98101.0,47.61393,-122.33810,NaN,2089.28,2.19
3,5,"{'latitude': '47.61421585', 'longitude': '-122...",1936.34,31.38,NaN,18081,98101.0,47.61412,-122.33664,NaN,286.43,4.67
4,8,"{'latitude': '47.6137544', 'longitude': '-122....",507.70,4.02,NaN,19576,98121.0,47.61375,-122.34047,NaN,505.01,2.88


En dehors des variables ZipCodes et ZipCode, il s'agit bien des mêmes variables. En fait la variable **Localisation** de 2015 contient toutes les variables suivantes :
- Latitude
- Longitude
- Adress
- City
- State
- ZipCode

Extrayons toutes ces variables de **Localisation**.

### Traitement de la variable localisation

Comme nous avons précédemment, plusieurs variables de 2016 sont réunis dans la variable *Localisation* sous forme de dictionnaire. Nous allons recréer ces variables en les extrayant de *Localisation*.

In [50]:
import ast

data_2015 = pd.concat([raw_data_2015.drop('Location', axis=1), # Delete 'Location' column
                       raw_data_2015['Location'].apply(ast.literal_eval) # Convert string into dictionary
                                                .apply(pd.Series)], # Split the dictionary into columns
                      axis=1)

Il reste encore à séparer les données de human_address en plusieurs colonnes.

In [51]:
data_2015 = pd.concat([data_2015.drop('human_address', axis=1), # Delete 'human_address' column
                       data_2015['human_address'].apply(ast.literal_eval) # Convert string into dictionary
                                                 .apply(pd.Series)], # Split the dictionary into columns
                      axis=1)

### Renommer les variables de 2015 comme celles de 2016

In [52]:
data_2015.rename(columns={'GHGEmissions(MetricTonsCO2e)': 'TotalGHGEmissions',
                  'GHGEmissionsIntensity(kgCO2e/ft2)': 'GHGEmissionsIntensity',
                  'Comment': 'Comments',
                  'latitude': 'Latitude',
                  'longitude': 'Longitude',
                  'address': 'Address',
                  'city': 'City',
                  'state': 'State',
                  'zip': 'ZipCode'},
                 inplace=True)

Il ne reste plus qu'à vérifier que toutes les variables de 2016 sont bien présentes dans le jeu de données de 2015.

In [53]:
features_2015 = data_2015.columns.tolist()
features_2016 = raw_data_2016.columns.tolist()
features = [c for c in features_2016 if c in features_2015]
print(len(features))

46


C'est bon, toutes les variables de 2016 sont bien présentes dans le jeu de données de 2015. Il nous reste maintenant à traiter les variables de 2015 non présentes dans le jeu de données de 2016.

### Variables non présentes dans le jeu de données 2016

In [54]:
features = [c for c in features_2015 if c not in features_2016]
print(features)

['OtherFuelUse(kBtu)', '2010 Census Tracts', 'Seattle Police Department Micro Community Policing Plan Areas', 'City Council Districts', 'SPD Beats', 'Zip Codes']


Voici la liste des variables manquantes :

Ayant accès à la localisation il est peu pertinent de les garder. C'est d'ailleurs le cas pour toutes les variables ci dessous :
- OtherFuelUse(kBtu)
- 2010 Census Tracts
- Seattle Police Department Micro Community Policing Plan Areas
- City Council Districts
- SPD Beats (ils s'agit de zone de délimitation, elles sont au nombre de cinq à Seattle)
- Zip Codes

Les cinq dernières ne devraient pas changer d'une année à l'autre, et peuvent donc être intégrée au jeu de données de 2016 en reprenant les même valeurs que celles de 2015.

En revanche, ce n'est pas le cas pour **OtherFuelUse(kBtu)**. Il faudra vérifier si on ne peut pas l'obtenir via d'autres variables. Si ce n'est pas le cas, alors il faudra voir si ses valeurs sont négligeables par rapport aux autres sources de consommation.

### Ajout des variables manquantes pour le jeu de données de 2016

In [55]:
feat_to_add =['OSEBuildingID', '2010 Census Tracts',
              'Seattle Police Department Micro Community Policing Plan Areas',
              'City Council Districts',
              'SPD Beats',
              'Zip Codes']

data_2016 = pd.merge(raw_data_2016, data_2015[feat_to_add], how='left', on='OSEBuildingID')

### Traitement de la variable OtherFuelUse(kBtu)

Nous allons commencer par regarder s'il est possible de calculer cette variable à partir de la consommation totale et des autres sources d'énergie.

Puis nous diviserons la différence entre le calcul et la variable *OtherFuelUse(kBtu)* par la consommation totale *SiteEnergyUse(kBtu)*. Cela permettra de vérifier si la différence est négligeable et donc de valider le calcul.

In [56]:
df = data_2015[['SiteEnergyUse(kBtu)', 'SiteEnergyUseWN(kBtu)',
                'SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)', 'OtherFuelUse(kBtu)']].copy()

df['Other_calc'] = df.iloc[:, 0] - df.iloc[:, 2:].sum(axis=1)
df['Other_calc'] = df['Other_calc'] / df.iloc[:, 0]

In [57]:
df.describe()

,SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),OtherFuelUse(kBtu),Other_calc
count,3.330000e+03,3.330000e+03,3.330000e+03,3.330000e+03,3.330000e+03,3.330000e+03,3328.000000
mean,4.983106e+06,5.203055e+06,2.502851e+05,3.473209e+06,1.253304e+06,7.142301e+03,-0.000246
std,1.375330e+07,1.400757e+07,3.481900e+06,9.519639e+06,4.415142e+06,1.962790e+05,0.007625
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.314228
25%,9.139300e+05,9.883248e+05,0.000000e+00,6.363322e+05,0.000000e+00,0.000000e+00,-0.000041
50%,1.776219e+06,1.953996e+06,0.000000e+00,1.160236e+06,2.945770e+05,0.000000e+00,-0.000033
75%,4.044277e+06,4.368462e+06,0.000000e+00,2.750847e+06,1.115366e+06,0.000000e+00,-0.000022
max,2.958126e+08,2.977410e+08,1.278697e+08,2.847263e+08,1.364484e+08,8.269669e+06,0.000001


Pour la majorité des bâtiments, la différence est marginale par rapport à la consommation totale (<0.1%). A noter que les valeurs obtenues après cacul sont sytématiquement plus faibles .

Nous observons de plus, que pour au moins une des observations, la différence est significative, représentant plus de 30% de la consommation totale.  
Vérifions s'il s'agit d'un cas isolé en affichant toutes les observations dont la différence représente plus de 1% de la consommation totale.

In [58]:
outliers = df[df['Other_calc'].abs()>0.001]
print(outliers.shape[0])
outliers

3


,SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),OtherFuelUse(kBtu),Other_calc
564,5288304.0,5288304.0,0.0,5288523.0,618364.0,0.0,-0.116972
1201,1554939.0,2129737.0,0.0,1482658.0,560886.0,0.0,-0.314228
1308,4217108.0,4217108.0,0.0,4217284.0,1201563.0,0.0,-0.284968


Ces valeurs anomaliques sont seulement au nombre de trois. En y regardant de plus près, nous notons que pour deux d'entre elles la consommation en électricité seule est supérieure à la consommation totale, ce qui est impossible. Il s'agit probablement d'outliers, vérifions s'ils ont été étiquetés comme tels grâce à la variable *Outlier*.

In [59]:
idx = outliers.index
data_2015.loc[idx, ['SiteEnergyUse(kBtu)', 'Electricity(kBtu)', 'Outlier']]

,SiteEnergyUse(kBtu),Electricity(kBtu),Outlier
564,5288304.0,5288523.0,NaN
1201,1554939.0,1482658.0,NaN
1308,4217108.0,4217284.0,NaN


Ces observations n'ont pas été étiquetées comme outliers, mais elles le sont néanmoins. Il y a nécessairement une erreur lors du repport de la consommation. Cette dernière étant une des valeurs cibles que l'on cherche à modéliser, il est préférable d'écarter ces *outliers*.

In [60]:
data_2015.drop(index=idx, inplace=True)

## Sélection des observations

### Vérifier la présence de doublons

Pour vérifier la présence de doublons, nous utiliserons la variable *OSEBuildingID*. Au vu de son nom, il s'agit certainement d'une clé primaire permettant d'identifier chaque bâtiment de manière unique, donc particulièrement adaptée pour la recherche de doublons.

Nous devons au préalable vérifier que toutes les données sont bien renseignées.

In [80]:
na_15 = data_2015[data_2015['OSEBuildingID'].isna()].size
na_16 = data_2015[data_2015['OSEBuildingID'].isna()].size
print("Il y a {} données manquante pour les données de 2015".format(na_15))
print("Il y a {} données manquante pour les données de 2016".format(na_16))


Il y a 0 données manquante pour les données de 2015
Il y a 0 données manquante pour les données de 2016


C'est bien le cas, nous pouvons donc utiliser cette variable comme identifiant pour vérifier la présence de doublons

In [81]:
n_dup_15 = data_2015.shape[0] - data_2015['OSEBuildingID'].unique().size
n_dup_16 = data_2016.shape[0] - data_2016['OSEBuildingID'].unique().size
print("Il y a {} doublons pour les données de 2015".format(n_dup_15))
print("Il y a {} doublons pour les données de 2016".format(n_dup_16))

Il y a 0 doublons pour les données de 2015
Il y a 0 doublons pour les données de 2016


Il n'y a pas de doublons pour les deux jeux de données, nous utiliserons la variable *OSEBuildingID* comme index.

In [84]:
data_2015.set_index('OSEBuildingID', inplace=True)
data_2016.set_index('OSEBuildingID', inplace=True)

### Traiter les outliers

Nous avons vu précédement qu'une variable *Oulier* permettait de renseigner si des observations étaient des outliers ou non. Vérifions si beaucoup d'outliers ont été identifiés.

In [61]:
data_2015['Outlier'].value_counts()

High Outlier    46
Low Outlier     38
Name: Outlier, dtype: int64

Au vu du faible nombre d'outliers identifiés, il est préférable de les écarter.

In [62]:
idx = data_2015[data_2015['Outlier'].notna()].index
data_2015.drop(index=idx, inplace=True)

### Écarter les bâtiments destinés à l'habitation

In [82]:
data_2015.columns

Index(['OSEBuildingID', 'DataYear', 'BuildingType', 'PrimaryPropertyType',
       'PropertyName', 'TaxParcelIdentificationNumber', 'CouncilDistrictCode',
       'Neighborhood', 'YearBuilt', 'NumberofBuildings', 'NumberofFloors',
       'PropertyGFATotal', 'PropertyGFAParking', 'PropertyGFABuilding(s)',
       'ListOfAllPropertyUseTypes', 'LargestPropertyUseType',
       'LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseType',
       'SecondLargestPropertyUseTypeGFA', 'ThirdLargestPropertyUseType',
       'ThirdLargestPropertyUseTypeGFA', 'YearsENERGYSTARCertified',
       'ENERGYSTARScore', 'SiteEUI(kBtu/sf)', 'SiteEUIWN(kBtu/sf)',
       'SourceEUI(kBtu/sf)', 'SourceEUIWN(kBtu/sf)', 'SiteEnergyUse(kBtu)',
       'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)', 'OtherFuelUse(kBtu)',
       'TotalGHGEmissions', 'GHGEmissionsIntensity', 'DefaultData', 'Comments',
       'ComplianceStatus', 'Outlier', '2010 Census Tracts',
       'Seattle Police Department Micro Community Policing Plan Area

In [83]:
data_2015['BuildingType'].value_counts()

NonResidential          1430
Multifamily LR (1-4)     996
Multifamily MR (5-9)     537
Multifamily HR (10+)     104
SPS-District K-12         97
Nonresidential COS        68
Campus                    21
Name: BuildingType, dtype: int64

In [87]:
996+537+104

1637

In [102]:
data_2015['PrimaryPropertyType'].value_counts()

Low-Rise Multifamily           965
Mid-Rise Multifamily           522
Small- and Mid-Sized Office    283
Other                          258
Non-Refrigerated Warehouse     187
Large Office                   160
K-12 School                    134
Mixed Use Property             125
High-Rise Multifamily          100
Retail Store                    96
Worship Facility                72
Hotel                           71
Distribution Center\n           51
Medical Office                  43
Senior Care Community           43
Supermarket/Grocery Store       36
Self-Storage Facility\n         27
College/University              22
Residence Hall/Dormitory        15
Refrigerated Warehouse          13
Restaurant\n                    11
Hospital                        10
SPS-District K-12                4
Distribution Center              2
Restaurant                       1
Self-Storage Facility            1
Laboratory                       1
Name: PrimaryPropertyType, dtype: int64

In [89]:
data_2015[data_2015['BuildingType']=='Multifamily LR (1-4)']['PrimaryPropertyType'].value_counts()

Low-Rise Multifamily           963
Senior Care Community           15
Mixed Use Property              12
Other                            4
College/University               1
Small- and Mid-Sized Office      1
Name: PrimaryPropertyType, dtype: int64

In [101]:
s_test = data_2015[data_2015['BuildingType']=='Multifamily LR (1-4)']['ListOfAllPropertyUseTypes'].dropna()
l_test = s_test.str.split(pat=", ").tolist()
flat_list = [i for sublist in l_test for i in sublist]
s = pd.Series(flat_list, name='PropertyUseTypes')
use_type = s.unique()
use_type

array(['Multifamily Housing', 'Retail Store', 'Medical Office', 'Office',
       'Parking', 'Other - Services', 'Restaurant', 'Other',
       'Non-Refrigerated Warehouse', 'Residence Hall/Dormitory',
       'Senior Care Community', 'Swimming Pool', 'Other - Restaurant/Bar',
       'Self-Storage Facility', 'Supermarket/Grocery Store',
       'Food Service', 'Other - Entertainment/Public Assembly',
       'Other - Recreation', 'Other - Education', 'Fast Food Restaurant',
       'Fitness Center/Health Club/Gym',
       'Personal Services (Health/Beauty', 'Dry Cleaning', 'etc)',
       'Other - Lodging/Residential', 'College/University',
       'Social/Meeting Hall', 'Museum', 'Bank Branch', 'Strip Mall',
       'Urgent Care/Clinic/Other Outpatient',
       'Hospital (General Medical & Surgical)'], dtype=object)

In [92]:
data_2015[data_2015['BuildingType']=='Multifamily LR (1-4)']['ListOfAllPropertyUseTypes'].value_counts()

Multifamily Housing                                                                                                                                                            584
Multifamily Housing, Parking                                                                                                                                                   229
Multifamily Housing, Retail Store                                                                                                                                               24
Multifamily Housing, Office, Parking                                                                                                                                            22
Multifamily Housing, Parking, Retail Store                                                                                                                                      12
Multifamily Housing, Office                                                                              

In [91]:
963+15+12+4+1+1

996

In [90]:
data_2015[data_2015['BuildingType']=='Multifamily LR (1-4)']['LargestPropertyUseType'].value_counts()

Multifamily Housing            931
Senior Care Community           14
Other                            3
College/University               1
Office                           1
Other - Lodging/Residential      1
Name: LargestPropertyUseType, dtype: int64

In [86]:
data_2015['LargestPropertyUseType'].value_counts()

Multifamily Housing                                     1547
Office                                                   454
Non-Refrigerated Warehouse                               194
K-12 School                                              134
Other                                                    104
Retail Store                                              94
Worship Facility                                          70
Hotel                                                     66
Distribution Center                                       52
Medical Office                                            43
Senior Care Community                                     42
Supermarket/Grocery Store                                 37
Parking                                                   28
Self-Storage Facility                                     27
College/University                                        21
Other - Entertainment/Public Assembly                     20
Other - Recreation      

In [ ]:
# data_2015[P]

## Nettoyage des variables liés à la consommation

### Vérifier que les consommations sont bien positives

In [63]:
col_conso = ['SiteEnergyUse(kBtu)',
             'SiteEnergyUseWN(kBtu)',
             'SteamUse(kBtu)', 'Electricity(kWh)',
             'Electricity(kBtu)',
             'NaturalGas(therms)',
             'NaturalGas(kBtu)',
             'OtherFuelUse(kBtu)']
data_2015[col_conso].describe()

,SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),OtherFuelUse(kBtu)
count,3.245000e+03,3.245000e+03,3.245000e+03,3.245000e+03,3.245000e+03,3.245000e+03,3.245000e+03,3.245000e+03
mean,4.945992e+06,5.164167e+06,2.440606e+05,1.012985e+06,3.456450e+06,1.238293e+04,1.238293e+06,7.329388e+03
std,1.383722e+07,1.409213e+07,3.489462e+06,2.803500e+06,9.565940e+06,4.451396e+04,4.451396e+06,1.988304e+05
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,9.181770e+05,9.927380e+05,0.000000e+00,1.880320e+05,6.415920e+05,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.758580e+06,1.930609e+06,0.000000e+00,3.400090e+05,1.160158e+06,2.861000e+03,2.861160e+05,0.000000e+00
75%,3.983784e+06,4.313191e+06,0.000000e+00,8.030860e+05,2.740242e+06,1.087800e+04,1.087808e+06,0.000000e+00
max,2.958126e+08,2.977410e+08,1.278697e+08,8.344505e+07,2.847263e+08,1.364484e+06,1.364484e+08,8.269669e+06


Les valeurs sont bien positives ou nulles

### Vérifier les conversions

En parcourant les variables de consommation disponibles, nous notons que certaines variables donnent la même information mais avec des unités différentes :
- Electricity(kBtu) et Electricity(kWh)
- 'NaturalGas(kBtu)' et 'NaturalGas(therms)'

L'unité usuelle pour *l'éléctricité* est le *kiloWatt heure (kWh)*. La consommation énergétique totale étant exprimée en *kBtu*, la consommation électrique mesurée en kWh a certainement été convertie en kBtu pour pouvoir la comparer à la consommation totale.
Si la conversion a bien été effectuée, le ratio consommation électrique en kWh sur consommation électrique en kBtu, devrait être le même pour toute les observations, ie. *3.41214*.

In [64]:
test_elec = data_2015['Electricity(kBtu)'] / data_2015['Electricity(kWh)']
test_elec.describe()

count    3243.000000
mean        3.412219
std         0.004470
min         3.411612
25%         3.412139
50%         3.412142
75%         3.412143
max         3.666667
dtype: float64

Si pour la majorité le résultat obtenu est très proche de la valeur exacte, il n'est pas le même pour toutes les obervations. Cela rajoute inutilement du bruit aux données et pourrait abaisser les performances du modèle, il est donc préférable de corriger cette variable.

In [65]:
data_2015['Electricity(kBtu)_corr'] = data_2015['Electricity(kBtu)'] * 3.41214

Regardons maintenant le *gaz naturel*.

Aux Etats-Unis, la quantité de gaz naturel consommé est exprimé en *therms* sur les factures. Là encore, cette valeur a été convertie en KBtu pour pouvoir la comparer à la consommation énergétique totale. Ici, la conversion est très simple vu qu'il suffit de multiplier par *100* pour passer des *therms* aux *kBtu*.
Verifions si la conversion a bien été effectuée.

In [66]:
test_gas = data_2015['NaturalGas(kBtu)'] / data_2015['NaturalGas(therms)']
test_gas.describe()

count    2014.000000
mean      100.003450
std         0.498022
min        91.250000
25%        99.997501
50%       100.000000
75%       100.002301
max       111.000000
dtype: float64

Nous observons comme pour l'électricité que la conversion n'a pas été bien effectuée pour toutes les données, là encore il faut corriger les donnée en utilisant le bon taux de conversion.

In [67]:
data_2015['NaturalGas(kBtu)_corr'] = data_2015['NaturalGas(therms)'] * 100

Il est en général préférable d'éviter d'avoir des variables redondantes, au risque sinon de voir des variables sur-représentés lors de la modélisation. Nous garderons que les consommations exprimées en kBtu (la version corrigée pour l'électricité et le gaz naturel)

In [68]:
col_to_del = ['Electricity(kBtu)', 'Electricity(kWh)', 'NaturalGas(kBtu)', 'NaturalGas(therms)']
data_2015.drop(columns=col_to_del, inplace=True)

### Vérification des totaux

## Nettoyage des variables liées au Gross Floor Area (GFA)

In [ ]:
Pour finir avec le nettoyages des variables nous regarderons si les totaux sont justes. Nous vérifierons notamment que :
- PropertyGFATotal = PropertyGFAParking + 

Même si la différence est faible entre la valeur calculée et la variable initiale *OtherFuelUse(kBtu)*, il est préférable d'avoir des données les plus propres possible pour la modélisation. Nous remplacerons cette dernière par la valeur calculée pour l'année 2015, et ajouterons cette variable calculée aux données de 2015.

In [31]:
data_2015['OtherFuelUse(kBtu)'] = data_2015['SiteEnergyUse(kBtu)'] - data_2015[['SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)']].sum(axis=1)

data_2016['OtherFuelUse(kBtu)'] = data_2016['SiteEnergyUse(kBtu)'] - data_2016[['SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)']].sum(axis=1)

In [30]:
data_2015[['SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)']].sum(axis=1)

0        6981580.0
1        8354396.0
2       73132739.0
3       28229435.0
4       14829350.0
           ...    
3335     1142996.0
3336     1543328.0
3337    16834654.0
3338      777096.0
3339      966416.0
Length: 3337, dtype: float64

In [32]:
data_2015[['SiteEnergyUse(kBtu)', 'SiteEnergyUseWN(kBtu)',
                'SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)', 'OtherFuelUse(kBtu)']]

,SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),OtherFuelUse(kBtu)
0,6981428.0,7097539.0,2023032.0,3686160.0,1272388.0,-152.0
1,8354235.0,8765788.0,0.0,3905411.0,4448985.0,-161.0
2,73130656.0,75506272.0,19660404.0,49762435.0,3709900.0,-2083.0
3,28229320.0,28363444.0,23458518.0,2769023.0,2001894.0,-115.0
4,14829099.0,15078243.0,0.0,6066245.0,8763105.0,-251.0
...,...,...,...,...,...,...
3335,1142949.0,1240429.0,0.0,1142996.0,0.0,-47.0
3336,1543282.0,1750143.0,0.0,1111352.0,431976.0,-46.0
3337,16834414.0,19623362.0,0.0,5767761.0,11066893.0,-240.0
3338,777064.0,849250.0,0.0,777096.0,0.0,-32.0


In [ ]:
df = data_2015[['SiteEnergyUse(kBtu)', 'SiteEnergyUseWN(kBtu)',
                'SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)', 'OtherFuelUse(kBtu)']].copy()

df['Other_calc'] = df.iloc[:, 0] - df.iloc[:, 2:].sum(axis=1)
df['Other_calc'] = df['Other_calc'] / df.iloc[:, 0]

In [18]:
test_elec = data_2015['Electricity(kBtu)'] / data_2015['Electricity(kWh)']
test_elec.describe()

count    3328.000000
mean        3.412217
std         0.004412
min         3.411612
25%         3.412139
50%         3.412142
75%         3.412143
max         3.666667
dtype: float64

In [19]:
test_gas = data_2015['NaturalGas(kBtu)'] / data_2015['NaturalGas(therms)']
test_gas.describe()

count    2074.000000
mean      100.027864
std         0.838721
min        91.250000
25%        99.997525
50%       100.000000
75%       100.002271
max       122.000000
dtype: float64

In [20]:
test_tot = data_2015['SiteEnergyUse(kBtu)'] - data_2015['SiteEUI(kBtu/sf)']*data_2015['PropertyGFABuilding(s)']
test_tot = test_tot/data_2015['SiteEnergyUse(kBtu)']
test_tot.describe()

count    3328.000000
mean       -0.021069
std         0.249590
min        -4.581702
25%        -0.002220
50%        -0.000047
75%         0.001617
max         2.295078
dtype: float64

In [21]:
test_tot[test_tot.abs()>0.01].size

1429

In [22]:
test_gfa = data_2015['PropertyGFATotal'] - (data_2015['PropertyGFAParking']+data_2015['PropertyGFABuilding(s)'])
test_gfa = test_tot/data_2015['PropertyGFATotal']
test_gfa.describe()

count    3.328000e+03
mean    -2.324305e-07
std      4.610394e-06
min     -8.238145e-05
25%     -6.007792e-08
50%     -7.064155e-10
75%      4.631160e-08
max      6.820846e-05
dtype: float64

In [23]:
data_2015['Outlier'].value_counts()

High Outlier    46
Low Outlier     38
Name: Outlier, dtype: int64

In [24]:
data_2015.columns

Index(['OSEBuildingID', 'DataYear', 'BuildingType', 'PrimaryPropertyType',
       'PropertyName', 'TaxParcelIdentificationNumber', 'CouncilDistrictCode',
       'Neighborhood', 'YearBuilt', 'NumberofBuildings', 'NumberofFloors',
       'PropertyGFATotal', 'PropertyGFAParking', 'PropertyGFABuilding(s)',
       'ListOfAllPropertyUseTypes', 'LargestPropertyUseType',
       'LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseType',
       'SecondLargestPropertyUseTypeGFA', 'ThirdLargestPropertyUseType',
       'ThirdLargestPropertyUseTypeGFA', 'YearsENERGYSTARCertified',
       'ENERGYSTARScore', 'SiteEUI(kBtu/sf)', 'SiteEUIWN(kBtu/sf)',
       'SourceEUI(kBtu/sf)', 'SourceEUIWN(kBtu/sf)', 'SiteEnergyUse(kBtu)',
       'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)', 'Electricity(kWh)',
       'Electricity(kBtu)', 'NaturalGas(therms)', 'NaturalGas(kBtu)',
       'OtherFuelUse(kBtu)', 'TotalGHGEmissions', 'GHGEmissionsIntensity',
       'DefaultData', 'Comments', 'ComplianceStatus', 'Outlier',
